## Generating Atoms Models

Importing SymbolInfo Class.

In [ ]:
from wizard.atoms import SymbolInfo

Creating bulk atoms.

In [ ]:
atoms = SymbolInfo('W', 'bcc', 3.185).create_bulk_atoms()

Importing Morph Class.

In [ ]:
from wizard.atoms import Morph

Creating HEA model.

In [ ]:
Morph(atoms).prop_element_set(['Mo','Nb','Ta','V','W'])

Creating mono-vacancy model.

In [ ]:
Morph(atoms).create_vacancy()

Creating di-vacancies model.

In [ ]:
Morph(atoms).create_divacancies()

Creating self-interstitial-atom model.

In [ ]:
Morph(atoms).create_self_interstitial_atom([1, 1, 1])

Creating interstitial-atom model.

In [ ]:
from ase import Atom
atom = Atom('W', position=(0, 0, 0))
atoms.append(atom)

Creating Frenkel defects model.

In [ ]:
Morph(atoms).create_fks(10)

## Molecular dynamics

Run molecular dynamics (MD) simulations using [GPUMD](https://github.com/brucefan1983/GPUMD).

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Running molecular dynamics simulations.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1', 
          'ensemble npt_scr 300 300 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_restart 30000', 
          'dump_exyz 10000',
          'run 30000']
Morph(atoms).gpumd('relax', run_in)

Deforming the simulation box.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 300', 
          'time_step 1',
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'run 30000', 
          'ensemble npt_scr 300 300 100 0 0 0 100 100 100 1000',
          'deform 0.00001 0 0 1', 
          'dump_thermo 1000', 
          'dump_exyz 1000', 
          'dump_restart 10000',
          'run 1000000']
Morph(atoms).gpumd('deform', run_in)

Simulating the process of a crystallization.

In [ ]:
run_in = ['potential ../nep.txt', 
          'velocity 2000', 
          'time_step 1', 
          'ensemble npt_scr 2000 2000 200 0 500 2000', 
          'dump_thermo 1000', 
          'dump_xyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 2000 5000 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 100000', 
          'dump_restart 10000', 
          'run 10000000',
          'ensemble npt_scr 4500 4500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 10000', 
          'dump_restart 10000', 
          'run 100000',
          'ensemble npt_scr 4500 1500 200 0 500 2000',
          'dump_thermo 1000', 
          'dump_xyz 100000', 
          'dump_restart 10000', 
          'run 10000000']
Morph(atoms).gpumd('crystallization', run_in)

Calculating the Melting point using two-phase coexistence method

In [ ]:
from wizard.io import read_xyz

group = []
for atom in atoms:
    if atom.position[2] < atoms.cell[2, 2] / 2:
        group.append(0)
    else:
        group.append(1)
atoms.info['group'] = group

run_in_1 = ['potential ../../nep.txt', 
            'velocity 3000', 
            'time_step 1', 
            'ensemble npt_ber 3000 3000 200 0 500 2000', 
            'dump_exyz 10000', 
            'dump_thermo 1000',
            'run 30000',
            'ensemble heat_lan 3500 200 500 0 1',
            'dump_exyz 10000',
            'dump_thermo 1000',
            'dump_restart 10000',
            'run 1000000']

#Morph(atoms).gpumd('melting_point/relax', run_in_1)

for Tm in range(3400, 3701, 100):
    atoms = read_xyz('melting_point/relax/dump.xyz')[-1]
    run_in = ['potential ../../nep.txt', 
             f'velocity {Tm}', 
              'time_step 1', 
             f'ensemble npt_ber {Tm} {Tm} 200 0 500 2000', 
              'dump_exyz 10000', 
              'dump_thermo 1000',
              'run 30000']
    Morph(atoms).gpumd(f'melting_point/{Tm}', run_in)

Simulation the radiation damage

In [ ]:
from wizard.io import read_xyz, dump_xyz
import numpy as np

group = []
thickness = 3.185 * 3
for atom in atoms:
    if atom.position[0] < thickness or atom.position[1] < thickness or atom.position[2] < thickness:
        group.append(0)
    elif atom.position[0] >= atoms.cell[0, 0] - thickness or atom.position[1] >= atoms.cell[1, 1] - thickness or atom.position[2] >= atoms.cell[2, 2] - thickness:
        group.append(1)
    else:
        group.append(2)
        
atoms.info['group'] = group
dump_xyz('test.xyz',atoms)

run_in_1 = ['potential ../../nep.txt',
            'velocity 300', 
            'time_step 1', 
            'ensemble npt_scr 300 300 200 0 500 2000', 
            'dump_thermo 1000', 
            'dump_restart 30000', 
            'run 30000']

run_in_2 = ['potential ../../nep.txt', 
            'velocity 300', 
            'time_step 0', 
            'ensemble nve',
            'dump_exyz 1', 
            'run 1',
            'time_step 1 0.015', 
            'ensemble heat_nhc 300 200 0 0 1',
            'electron_stop ../../electron_stopping_fit.txt',
            'compute 0 200 10 temperature', 
            'dump_restart 10000', 
            'dump_exyz 2000 1 1',
            'run 70000']

pka_energy = 10 #eV
angle = np.array([1, 3, 5]) 
index = 100

Morph(atoms).gpumd('radiation/relax', run_in_1)
atoms = read_xyz('radiation/relax/restart.xyz')[-1]
Morph(atoms).set_pka(pka_energy, angle, index)
Morph(atoms).gpumd('radiation/cascade', run_in_2)

## Calculating Material Properties

Using the calculator from [PyNEP](https://github.com/bigd4/PyNEP).

In [ ]:
from wizard.calculator import MaterialCalculator
from pynep.calculate import NEP

calc = NEP('../Repository/Wnep2/nep.txt')
material_calculator = MaterialCalculator(atoms, calc, 'W', 'bcc')

Calculate the lattice constants of the material, write them along with the atom energy to a file, and return the atom energy and cell lengths.

In [ ]:
material_calculator.lattice_constant()

Calculate the elastic constants of the material, write them to a file, and return the bulk modulus.

In [ ]:
material_calculator.elastic_constant()

Generate the equation of state (EOS) curve for the material, save the curve as a PNG image, and write the volume and energy data to a file.

In [ ]:
material_calculator.eos_curve()

Calculate and plot the phonon dispersion band structure for the material.

In [ ]:
material_calculator.phonon_dispersion()

Calculate the formation energy of a vacancy in the material, write it to a file, and return the formation energy.

In [ ]:
material_calculator.formation_energy_vacancy()

Calculate the migration energy of a vacancy in the material by simulating the transition from the initial to the final state.

In [ ]:
material_calculator.migration_energy_vacancy()

Calculate the formation energy of divacancies in the material, write it to a file, and return the formation energy.

In [ ]:
nth = 1
material_calculator.formation_energy_divacancies(nth)

Calculate the formation energy of a self-interstitial atom (SIA) in the material, write it to a file, and return the formation energy.

In [ ]:
vector = (1,1,1)
material_calculator.formation_energy_sia(vector)

Calculate the formation energy of an interstitial atom in the material, write it to a file, and return the formation energy.

In [ ]:
material_calculator.formation_energy_interstitial_atom('W',[0,0,1/2],'octahedral')

Calculate the formation energy of a surface in the material, write it to a file, and return the formation energy in meV.

In [ ]:
miller = (0, 0, 1)
material_calculator.formation_energy_surface(miller)

Calculate the energy of a stacking fault in the material by simulating the shift of atomic layers, write the maximum energy to a file, plot the energy as a function of shift distance, and return the energy values.

In [ ]:
material_calculator.stacking_fault(a = (1,1,-1), b = (1,-1,0), miller = [1,1,2], distance = 3.185/2)

Simulate the movement of a screw dislocation dipole in a pure BCC metal, calculate the energy at each step, save the energy profile as a PNG image, and return the energy values.

In [ ]:
material_calculator.pure_bcc_metal_screw_dipole_move()

Simulate the movement of a single screw dislocation in a pure BCC metal, calculate the energy at each step, save the energy profile as a PNG image, and return the energy values.

In [ ]:
material_calculator.pure_bcc_metal_screw_one_move()